# Projeto 1 - Ciência dos Dados

Nome: _____

Nome: _____

Atenção: Serão permitidos grupos de três pessoas, mas com uma rubrica mais exigente. Grupos deste tamanho precisarão fazer um questionário de avaliação de trabalho em equipe

___
Carregando algumas bibliotecas:

In [1]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
import re

In [2]:
print('Esperamos trabalhar no diretório')
print(os.getcwd())

Esperamos trabalhar no diretório
/Users/rodrigomedeiros/Documents/Insper/2_semestre/CienciaDosDados/Projeto1_CDados


Carregando a base de dados com os tweets classificados como relevantes e não relevantes:

In [3]:
filename = 'vacina.xlsx'

In [4]:
train = pd.read_excel(filename, sheet_name = 'Treinamento')
train.head(5)

,Treinamento,Classificação
0,@borboletadirceu as grávidas podem tomar vacin...,1
1,"com todo respeito, uma decisão quebra a fila ú...",0
2,minha tia tava falando q não ia tomar vacina (...,0
3,"adivinha, com lockdown e vacina https://t.co/f...",0
4,@revistaoeste olha as ideia... o povo achando ...,1


In [5]:
test = pd.read_excel(filename, sheet_name = 'Teste')
test.head(5)

,Teste,Classificação
0,@drunkclurichaun @folha o governo não compra p...,1
1,@mrnamelesszero @gentedemal a vacina é igual p...,1
2,@fiscaldoibama o judiciário ainda vai pedir au...,0
3,eu depois da vacina. https://t.co/vnoryzfqjy,0
4,mês q vem já tenho vacina pra toma dnv 😂🤦🏻‍♀️ ...,0


In [6]:
train['Classificação']= train["Classificação"].astype(str)

Relevante = train.loc[(train.Classificação=='1')]
Irrelevante = train.loc[(train.Classificação=='0')]

Relevantes = ''.join(Relevante.Treinamento)
Irrelevantes = ''.join(Irrelevante.Treinamento)

In [7]:
# Limpando as strings


def cleanup1(text):
    
    "Troca sinais e links por espaços"
    
    text = re.sub(r'@\S+', ' ', text, flags=re.MULTILINE)
    text = re.sub(r'_', ' ', text, flags=re.MULTILINE)
    text = re.sub(r'\n', ' ', text, flags=re.MULTILINE)
    text = re.sub(r'kk\S+', ' ', text, flags=re.MULTILINE)
    text = re.sub(r'http\S+', ' ', text, flags=re.MULTILINE)
    text = re.sub(r'https\S+', ' ', text, flags=re.MULTILINE)
    punctuation = '[!-.:?;_“”]'

    pattern = re.compile(punctuation)
    text_subbed = re.sub(pattern, ' ', text)
    return text_subbed

emoji_pat = '[\U0001F300-\U0001F64F\U0001F680-\U0001F6FF\u2600-\u26FF\u2700-\u27BF]'
shrink_whitespace_reg = re.compile(r'\s{2,}')


def cleanup2(text_subbed):
    
    "Tira Espaços em excesso e separa emojis com espaços"
    
    reg = re.compile(r'({})'.format(emoji_pat))
    result = reg.sub(lambda x: ' {} '.format(x.group(1)) if x.group(1) else ' ', text_subbed)
    return shrink_whitespace_reg.sub(' ', result)

In [8]:
Relevantes_clean = cleanup1(Relevantes)
Relevantes_clean = cleanup2(Relevantes_clean)

Irrelevantes_clean = cleanup1(Irrelevantes)
Irrelevantes_clean = cleanup2(Irrelevantes_clean)

In [9]:
Palavras_relevantes = pd.Series(Relevantes_clean.split())


Palavras_irrelevantes = pd.Series(Irrelevantes_clean.split())

print('Número de palavras relevantes:')
print(len(Palavras_relevantes))
print('\n')
print('Número de palavras irrelevantes:')
print(len(Palavras_irrelevantes))

Número de palavras relevantes:
2538


Número de palavras irrelevantes:
2336


In [10]:
lista_total_palavras = []

for x in Palavras_relevantes:
    lista_total_palavras.append(x)
    
for y in Palavras_irrelevantes:
    lista_total_palavras.append(y)

total_palavras_n_repitir = pd.Series(lista_total_palavras)

In [11]:
total_lista_absoluta = total_palavras_n_repitir.value_counts(ascending=False)

len(total_lista_absoluta)

1620

In [12]:
tabela_absoluta_relevantes = Palavras_relevantes.value_counts(ascending=False)


tabela_absoluta_relevantes

vacina    99
de        94
a         86
o         65
não       64
          ..
doente     1
mão        1
fica       1
d          1
num        1
Length: 959, dtype: int64

In [13]:
tabela_absoluta_irrelevantes = Palavras_irrelevantes.value_counts(ascending=False)


tabela_absoluta_irrelevantes

vacina      114
a            97
de           73
que          57
e            56
           ... 
gênio         1
grande        1
acaso         1
judicial      1
levar         1
Length: 975, dtype: int64

In [14]:
tabela_relativa_relevantes = Palavras_relevantes.value_counts(True)
tabela_relativa_relevantes.sum()

1.0

In [15]:
tabela_relativa_irrelevantes = Palavras_irrelevantes.value_counts(True)
tabela_relativa_irrelevantes.sum()

1.0

___
## Classificador automático de sentimento


Faça aqui uma descrição do seu produto e o que considerou como relevante ou não relevante na classificação dos tweets.

O produto escolhido foi "vacina", tomei como relevantes os tweets que se posicionam a favor ou contra da vacina e como irrelevantes os que não demonstram um posicionamento.

___
### Montando um Classificador Naive-Bayes

Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.

In [16]:
# total de palavras
total = len(Palavras_relevantes) + len(Palavras_irrelevantes)

# Probabilidade de ser relevantes:
total_relevantes = len(Palavras_relevantes)/total

# Probabilidade de ser irrelevantes:
total_irrelevantes = len(Palavras_irrelevantes)/total

#total de palavras sem repitir
total_lista_absoluta

print(total_relevantes)
print(total_irrelevantes)

0.5207221994255232
0.47927780057447683


In [17]:
classificador = []

for tweet in test.Teste:
    y = cleanup1(tweet.lower())
    tweets = cleanup2(y)
    r = 1
    i = 1
 
    for x in tweets:
        
        if x not in tabela_absoluta_relevantes:
            r = r * (0 + 1)/(len(Palavras_relevantes) + len(total_lista_absoluta))
        else:
            r = r *  (tabela_absoluta_relevantes[x] + 1)/(len(Palavras_relevantes) + len(total_lista_absoluta))
        if x not in tabela_absoluta_irrelevantes:
            i = i * (0 + 1)/(len(Palavras_irrelevantes) + len(total_lista_absoluta))
        else:
            i = i * (tabela_absoluta_irrelevantes[x] + 1)/(len(Palavras_irrelevantes) + len(total_lista_absoluta)) 
            
    R = total_relevantes * r 
    I = total_irrelevantes * i 
    
    
    if I > R:
        classificador.append("Irrelevante")
    else: 
        classificador.append("Relevante")
        

In [18]:
test['Classificador'] = classificador
test

,Teste,Classificação,Classificador
0,@drunkclurichaun @folha o governo não compra p...,1,Irrelevante
1,@mrnamelesszero @gentedemal a vacina é igual p...,1,Relevante
2,@fiscaldoibama o judiciário ainda vai pedir au...,0,Irrelevante
3,eu depois da vacina. https://t.co/vnoryzfqjy,0,Irrelevante
4,mês q vem já tenho vacina pra toma dnv 😂🤦🏻‍♀️ ...,0,Irrelevante
...,...,...,...
195,resumo do dia: bolsotario dizendo as suas ment...,0,Relevante
196,"@claudemirsl @biakicis nossa, você perdeu a ed...",0,Relevante
197,"no brasil, mais de 10 milhões de pessoas se cu...",1,Relevante
198,o juiz ao entender inconstitucionalidade na le...,1,Relevante


___
### Verificando a performance do Classificador

Agora você deve testar o seu classificador com a base de Testes.

In [19]:
pd.crosstab(test['Classificação'], test['Classificador'], normalize='columns').round(4)

Classificador,Irrelevante,Relevante
Classificação,,
0,0.6705,0.2857
1,0.3295,0.7143


In [20]:
acertos = (.7143 + .6705)/2
erros = (.3295 + .2857)/2

print("Verdadeiro positivo: 71.43%")
print("Falso positivo: 32.95%")
print("Verdadeiros negativos: 67.05%")
print("Falsos negativos: 28.57%")
print('')
print("Acertos: {:.2%}".format(acertos))
print("Erros: {:.2%}".format(erros))

Verdadeiro positivo: 71.43%
Falso positivo: 32.95%
Verdadeiros negativos: 67.05%
Falsos negativos: 28.57%

Acertos: 69.24%
Erros: 30.76%


___
### Concluindo

É possível concluir que o classificador está funcionando corretamente, pois, como mostrado acima, tivemos 69.24% de acertos, porém os tweets com dupla negação e sarcasmo acabam não sendo interpretados da forma que deveriam ser porque o classificador não leva em conta a frase na qual as palavras estão.
O plano de expansão é levar em consideração dupla negação e sarcasmo fazendo com que o classificar não olhe somente as palavras individualmente mas também as frases que elas compõem. Além disso é possível aumentar o número de grupos de classificação, como por exemplo muito relevantes e muito irrelevantes.

___
### Qualidade do Classificador a partir de novas separações dos tweets entre Treinamento e Teste

Caso for fazer esse item do Projeto

In [21]:
del test['Classificador']

In [22]:
test['tweet'] = test['Teste']
train['tweet'] = train['Treinamento']

del test['Teste']
del train['Treinamento']

In [24]:
todos_tweets = pd.concat([test, train])

rows_teste = round((200/500) * len(todos_tweets))
rows_treinamento = round(len(todos_tweets) - rows_teste)

indx = 0

for i in range (100):
    
    if indx == 1:
        del teste
        del treinamento
    
    teste = pd.DataFrame()
    treinamento = pd.DataFrame()
    
    transfer_teste = todos_tweets.iloc[- rows_teste:, 0:]
    teste = teste.append(transfer_teste)
    transfer_treinamento = todos_tweets.iloc[rows_treinamento:, 0:]
    treinamento = teste.append(transfer_treinamento)
    
    Relevante = treinamento.loc[(treinamento.Classificação=='1')]
    Irrelevante = treinamento.loc[(treinamento.Classificação=='0')]

    Relevantes = ''.join(Relevante)
    Irrelevantes = ''.join(Irrelevante)

    Relevantes_clean = cleanup1(Relevantes)
    Relevantes_clean = cleanup2(Relevantes_clean)

    Irrelevantes_clean = cleanup1(Irrelevantes)
    Irrelevantes_clean = cleanup2(Irrelevantes_clean)

    Palavras_relevantes = pd.Series(Relevantes_clean.split())
    Palavras_irrelevantes = pd.Series(Irrelevantes_clean.split())
    
    total = len(Palavras_relevantes) + len(Palavras_irrelevantes)
    total_relevantes = len(Palavras_relevantes)/total
    total_irrelevantes = len(Palavras_irrelevantes)/total
    
    lista_total_palavras = []

    for x in Palavras_relevantes:
        lista_total_palavras.append(x)
    
    for y in Palavras_irrelevantes:
        lista_total_palavras.append(y)

    total_palavras_n_repitir = pd.Series(lista_total_palavras)
    total_lista_absoluta = total_palavras_n_repitir.value_counts(ascending=False)
    
    classificador = []

    for tweet in teste.tweet:
        y = cleanup1(tweet.lower())
        tweets = cleanup2(y)
        r = 1
        i = 1
 
    for x in tweets:
        
        if x not in tabela_absoluta_relevantes:
            r = r * (0 + 1)/(len(Palavras_relevantes) + len(total_lista_absoluta))
        else:
            r = r *  (tabela_absoluta_relevantes[x] + 1)/(len(Palavras_relevantes) + len(total_lista_absoluta))
        if x not in tabela_absoluta_irrelevantes:
            i = i * (0 + 1)/(len(Palavras_irrelevantes) + len(total_lista_absoluta))
        else:
            i = i * (tabela_absoluta_irrelevantes[x] + 1)/(len(Palavras_irrelevantes) + len(total_lista_absoluta)) 
            
    R = total_relevantes * r 
    I = total_irrelevantes * i 
    
    
    if I > R:
        classificador.append("Irrelevante")
    else: 
        classificador.append("Relevante")

    counter_classificador = Counter()
    verdadeiro_positivo = (1, d(1))/(1, d(1))
    
    indx = 1
    
    teste['Classificador'] = classificador

Número de palavras relevantes:
1


Número de palavras irrelevantes:
1


ValueError: Length of values does not match length of index

___
## Aperfeiçoamento:

Os trabalhos vão evoluir em conceito dependendo da quantidade de itens avançados:

* Limpar: \n, :, ", ', (, ), etc SEM remover emojis
* Corrigir separação de espaços entre palavras e emojis ou entre emojis e emojis
* Propor outras limpezas e transformações que não afetem a qualidade da informação ou classificação
* Criar categorias intermediárias de relevância baseadas na probabilidade: ex.: muito relevante, relevante, neutro, irrelevante, muito irrelevante (3 categorias: C, mais categorias conta para B)
* Explicar por que não posso usar o próprio classificador para gerar mais amostras de treinamento
* Propor diferentes cenários para Naïve Bayes fora do contexto do projeto
* Sugerir e explicar melhorias reais com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa)


___
## Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**